In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 63.3 MB/s 


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#PyKoSpacing
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-ckqne5xc
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-ckqne5xc
     |████████████████████████████████| 671.4 MB 1.6 kB/s 
     |████████████████████████████████| 1.3 MB 10.6 MB/s 
     |████████████████████████████████| 463 kB 58.3 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268638 sha256=37c41405ce4192e659db2ca35d10fd2b2caee32f79b37ddd0f7a10c62316e806
  Stored in directory: /tmp/pip-ephem-wheel-cache-jc232la6/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempt

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 4.7 MB 16.5 MB/s 
     |████████████████████████████████| 1.3 MB 53.4 MB/s 
     |████████████████████████████████| 120 kB 54.8 MB/s 
     |████████████████████████████████| 6.6 MB 39.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=56e48dd5c3fd7a9e7e30bb39fc6e17292fe2f8ce6271d89e641889503ba5020b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import math
from collections import Counter
import itertools

import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

import re
import urllib.request
from konlpy.tag import Okt, Komoran, Hannanum, Kkma
from nltk.tokenize import word_tokenize, sent_tokenize

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices()[1])

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics.pairwise import cosine_similarity

from pykospacing import Spacing
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 단어 전처리

In [ ]:
with open("/content/drive/Shareddrives/22-여름 NLP 프로젝트/비상선언+외계인/비상선언_관객평점.pickle","rb") as fr:
    text1 = pickle.load(fr)
text1.head()

,평점,리뷰,공감,비공감
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0
3,1,,0,0
4,1,후반부 너무 질질끄네요..,0,0


In [ ]:
with open('/content/drive/Shareddrives/22-여름 NLP 프로젝트/NLP프로젝트/korean_stopwords.txt', encoding='utf-8') as f:
  stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
stopwords.extend(['하다','이다','이에요','은','한','거','던','라','고','비상','선언','영화'])
stopwords.extend("아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 들자면 저 소인 소생 저희 다른 물론 또한 그리고 비길수 해서는 만이 막론하고 관계없이 그치지 타다 오르다 제외하고 하여야 비로소 한다면 몰라도 이곳 여기 부터 기점으로 따라서 이리하여 그리하여 그렇게 함으로써 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 임에 한다면 등 등등 등등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 이외에도 대해 말하자면 뿐이다 다음에 말하자면 이와 바꾸어서 말하면 한다면 만약 까악 툭 딱 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 어떤 내 경우 명 생각 다시 이런 앞 보이 번 나 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 통하 놓 ㅋ ㅋㅋ ㅋㅋㅋ ㅋㅋㅋㅋ ㅋㅋㅋㅋㅋ ㅋㅋㅋㅋㅋㅋ ㅎ ㅎㅎ ㅎㅎㅎ ㅎㅎㅎㅎ ㅎㅎㅎㅎㅎ ㅎㅎㅎㅎㅎㅎ ㅠ ㅠㅠ ㅠㅠㅠ ㅠㅠㅠㅠ ㅠㅠㅠㅠㅠ ㅠㅠㅠㅠㅠㅠ . .. ... .... ..... ...... / - ' ? ! ?? ??? ???? ????? ?! !? !! !!! !!!! !!!!! ^^ . , # $ % \ ' ( ) * + - / : ; < = > @ [ \ \ ] ^ _ ` { | } ~ " " “ ” ’ ".split())
spacing = Spacing()

print(len(stopwords))

def preprocessing(review,okt,remove_stopwords=False,stop_words=[]):

  review = spacing(review)
  review_text = re.sub('[^ㄱ-ㅣ가-힣]+',' ',review)   
  # word_tokenize
  word_review = okt.morphs(review_text,stem=True)
  word_review_noun = ' '.join([word[0] for word in okt.pos(review_text) if word[1] == 'Noun'])
  # stop_words
  if remove_stopwords:
    word_review = [token for token in word_review if not token in stop_words]
  return word_review, word_review_noun

928


In [ ]:
# okt(), stopword 처리한 토큰
stop_words = stopwords
okt = Okt()
clean_review = []
noun_review = []
for review in tqdm(text1['리뷰']):
  if type(review) == str:
    clean, noun = preprocessing(review,okt,remove_stopwords=True,stop_words=stopwords)
    clean_review.append(clean)
    noun_review.append(noun)
  else:
    clean_review.append([])
    noun_review.append([])

text1_pp = text1.copy()
text1_pp['전처리1'] = clean_review

text1_pp['전처리2'] = ''
for i in range(len(text1_pp)):
    text1_pp['전처리2'][i] = ' '.join(text1_pp['전처리1'][i])

text1_pp['명사'] = noun_review

text1_pp.head()

100%|██████████| 5525/5525 [00:52<00:00, 105.54it/s]


,평점,리뷰,공감,비공감,전처리1,전처리2,명사
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,"[고증, 없다, 막장, 이건, 정말, 아니다]",고증 없다 막장 이건 정말 아니다,고증 막장 영화 이건 정말
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,"[아직도, 이렇다, 억지스럽다, 반일, 반미, 감정, 만, 부추기다, 개봉, 답, 없다]",아직도 이렇다 억지스럽다 반일 반미 감정 만 부추기다 개봉 답 없다,반일 반미 감정 영화 개봉 답
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,"[감독, 님, 공, 부안, 하고, 찍다, ㅋㅋㅋ]",감독 님 공 부안 하고 찍다 ㅋㅋㅋ,감독 부안
3,1,,0,0,[],,
4,1,후반부 너무 질질끄네요..,0,0,"[후반, 부, 너무, 질질, 끄다]",후반 부 너무 질질 끄다,후반 부 질질


In [ ]:
n_gram_range = (2,3) # Bigram, Trigram Select

text1_pp['키워드'] = ''
for i in range(len(text1_pp)):
    try:
        count = CountVectorizer(ngram_range=n_gram_range).fit([text1_pp['전처리2'][i]])
        text1_pp['키워드'][i] = count.get_feature_names_out()
    except:
        text1_pp['키워드'][i] = []

text1_pp

,평점,리뷰,공감,비공감,전처리1,전처리2,명사,키워드
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,"[고증, 없다, 막장, 이건, 정말, 아니다]",고증 없다 막장 이건 정말 아니다,고증 막장 영화 이건 정말,"[고증 없다, 고증 없다 막장, 막장 이건, 막장 이건 정말, 없다 막장, 없다 막..."
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,"[아직도, 이렇다, 억지스럽다, 반일, 반미, 감정, 만, 부추기다, 개봉, 답, 없다]",아직도 이렇다 억지스럽다 반일 반미 감정 만 부추기다 개봉 답 없다,반일 반미 감정 영화 개봉 답,"[감정 부추기다, 감정 부추기다 개봉, 개봉 없다, 반미 감정, 반미 감정 부추기다..."
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,"[감독, 님, 공, 부안, 하고, 찍다, ㅋㅋㅋ]",감독 님 공 부안 하고 찍다 ㅋㅋㅋ,감독 부안,"[감독 부안, 감독 부안 하고, 부안 하고, 부안 하고 찍다, 찍다 ㅋㅋㅋ, 하고 ..."
3,1,,0,0,[],,,[]
4,1,후반부 너무 질질끄네요..,0,0,"[후반, 부, 너무, 질질, 끄다]",후반 부 너무 질질 끄다,후반 부 질질,"[너무 질질, 너무 질질 끄다, 질질 끄다, 후반 너무, 후반 너무 질질]"
...,...,...,...,...,...,...,...,...
5520,5,,15,5,[],,,[]
5521,5,킬링타임용 K무비연기는 좋음,8,4,"[킬링타임, 용, 무비, 연기, 는, 좋다]",킬링타임 용 무비 연기 는 좋다,킬링타임 용 무비 연기,"[무비 연기, 무비 연기 좋다, 연기 좋다, 킬링타임 무비, 킬링타임 무비 연기]"
5522,5,기대가 실망의 끝자락을 잡고 급한데 화장실은 안보이고 결국에 바지에 실례한 기분,12,3,"[기대, 실망, 끝, 자락, 잡고, 급하다, 화장실, 안보, 이고, 바지, 실례, 기분]",기대 실망 끝 자락 잡고 급하다 화장실 안보 이고 바지 실례 기분,기대 실망 끝 자락 잡고 화장실 안보 바지 실례 기분,"[급하다 화장실, 급하다 화장실 안보, 기대 실망, 기대 실망 자락, 바지 실례, ..."
5523,5,생각보단 괜찮았음..,29,28,"[생각, 보단, 괜찮다]",생각 보단 괜찮다,생각,"[보단 괜찮다, 생각 보단, 생각 보단 괜찮다]"


# KeyBERT

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
text2_pp_v1 = text1.copy()

text2_pp_v1['키워드TOP3'] = ''
for i in tqdm(range(len(text2_pp_v1))):
    try:
        doc_embedding = model.encode([text1_pp['리뷰'][i]])
        candidate_embeddings = model.encode(text1_pp['키워드'][i])
        top_n = 3
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [text2_pp_v1['키워드'][i][index] for index in distances.argsort()[0][-top_n:]]
        text2_pp_v1['키워드TOP3'][i] = keywords
    except: pass
text2_pp_v1

100%|██████████| 5525/5525 [08:47<00:00, 10.47it/s]


,평점,리뷰,공감,비공감,키워드TOP3
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,
3,1,,0,0,
4,1,후반부 너무 질질끄네요..,0,0,
...,...,...,...,...,...
5520,5,,15,5,
5521,5,킬링타임용 K무비연기는 좋음,8,4,
5522,5,기대가 실망의 끝자락을 잡고 급한데 화장실은 안보이고 결국에 바지에 실례한 기분,12,3,
5523,5,생각보단 괜찮았음..,29,28,


In [ ]:
text2_pp_v2 = text2_pp_v1.copy()

text2_pp_v2['문장길이'] = text2_pp_v2['리뷰'].apply(lambda x: len(x.split()))
text2_pp_v2.drop_duplicates(['리뷰'],keep='last',inplace=True)
text2_pp_v2 = text2_pp_v2[text2_pp_v2['문장길이']>=3].reset_index(drop=True)
text2_pp_v2

,평점,리뷰,공감,비공감,키워드TOP3,문장길이
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,,7
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,,9
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,,3
3,1,후반부 너무 질질끄네요..,0,0,,3
4,1,재난영화가 아니라 재난 그자체인 영화다. 내가 아는 한재림감독이 맞나? 동명이인?!,1,1,,10
...,...,...,...,...,...,...
4679,5,아 시바 성무로 간다..,33,20,,4
4680,5,긴장감 잘 쌓아놓은 빌드업가지고 신파로 전부 말아먹는것도 재능..,5,1,,8
4681,5,"이 영화를 무조건적인 ""비상선언""을 선포합니다.",92,41,,5
4682,5,킬링타임용 K무비연기는 좋음,8,4,,3


In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
text3_pp_v1 = text1.copy()

text3_pp_v1['키워드TOP3_ lowDIV'] = ''
text3_pp_v1['키워드TOP3_ highDIV'] = ''

for i in tqdm(range(len(text3_pp_v1))):
    try:
        doc_embedding = model.encode([text1_pp['리뷰'][i]])
        candidates = text1_pp['키워드'][i]
        candidate_embeddings = model.encode(candidates)
        top_n = 4
        low_diverse = mmr(doc_embedding, candidate_embeddings, candidates, top_n=top_n, diversity=0.2)
        high_diverse = mmr(doc_embedding, candidate_embeddings, candidates, top_n=top_n, diversity=0.7)
        text3_pp_v1['키워드TOP3_ lowDIV'][i] = low_diverse
        text3_pp_v1['키워드TOP3_ highDIV'][i] = high_diverse
    except: pass

text3_pp_v1.head()

100%|██████████| 5525/5525 [35:06<00:00,  2.62it/s] 


,평점,리뷰,공감,비공감,키워드TOP3_ lowDIV,키워드TOP3_ highDIV
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,"[고증 없다 막장, 이건 정말 아니다, 없다 막장, 정말 아니다]","[고증 없다 막장, 정말 아니다, 없다 막장 이건, 이건 정말 아니다]"
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,"[아직도 이렇다 억지스럽다, 반미 감정 부추기다, 억지스럽다 반일 반미, 억지스럽다...","[아직도 이렇다 억지스럽다, 감정 부추기다, 반일 반미, 반미 감정 부추기다]"
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,"[감독 부안 하고, 하고 찍다 ㅋㅋㅋ, 부안 하고 찍다, 하고 찍다]","[감독 부안 하고, 감독 부안, 찍다 ㅋㅋㅋ, 하고 찍다 ㅋㅋㅋ]"
3,1,,0,0,,
4,1,후반부 너무 질질끄네요..,0,0,"[후반 너무, 후반 너무 질질, 너무 질질, 너무 질질 끄다]","[후반 너무, 질질 끄다, 너무 질질 끄다, 후반 너무 질질]"


In [ ]:
text3_pp_v2 = text3_pp_v1.copy()

text3_pp_v2['문장길이'] = text3_pp_v2['리뷰'].apply(lambda x: len(x.split()))
text3_pp_v2.drop_duplicates(['리뷰'],keep='last',inplace=True)
text3_pp_v2 = text3_pp_v2[text3_pp_v2['문장길이']>=3].reset_index(drop=True)
text3_pp_v2

,평점,리뷰,공감,비공감,키워드TOP3_ lowDIV,키워드TOP3_ highDIV,문장길이
0,1,고증 없는 막장 영화. 이건 정말 아닌듯,0,0,"[고증 없다 막장, 이건 정말 아니다, 없다 막장, 정말 아니다]","[고증 없다 막장, 정말 아니다, 없다 막장 이건, 이건 정말 아니다]",7
1,1,아직도 이런 억지스러운 반일 반미 감정만 부추기는 영화들이 개봉하는게...답없다,0,0,"[아직도 이렇다 억지스럽다, 반미 감정 부추기다, 억지스럽다 반일 반미, 억지스럽다...","[아직도 이렇다 억지스럽다, 감정 부추기다, 반일 반미, 반미 감정 부추기다]",9
2,1,감독님이 공부안하고 찍음ㅋㅋㅋ,0,0,"[감독 부안 하고, 하고 찍다 ㅋㅋㅋ, 부안 하고 찍다, 하고 찍다]","[감독 부안 하고, 감독 부안, 찍다 ㅋㅋㅋ, 하고 찍다 ㅋㅋㅋ]",3
3,1,후반부 너무 질질끄네요..,0,0,"[후반 너무, 후반 너무 질질, 너무 질질, 너무 질질 끄다]","[후반 너무, 질질 끄다, 너무 질질 끄다, 후반 너무 질질]",3
4,1,재난영화가 아니라 재난 그자체인 영화다. 내가 아는 한재림감독이 맞나? 동명이인?!,1,1,"[감독 맞다 동명이인, 알다 한재림 감독, 재난영화 아니다 재난, 재난 자체 알다]","[감독 맞다 동명이인, 한재림 감독, 재난 자체, 재난영화 아니다 재난]",10
...,...,...,...,...,...,...,...
4679,5,아 시바 성무로 간다..,33,20,,,4
4680,5,긴장감 잘 쌓아놓은 빌드업가지고 신파로 전부 말아먹는것도 재능..,5,1,"[빌드업가지 신파 말아먹다, 긴장감 자다, 긴장감 자다 쌓다, 말아먹다 재능]","[빌드업가지 신파 말아먹다, 쌓다 빌드업가지, 신파 말아먹다 재능, 긴장감 자다 쌓다]",8
4681,5,"이 영화를 무조건적인 ""비상선언""을 선포합니다.",92,41,,,5
4682,5,킬링타임용 K무비연기는 좋음,8,4,"[연기 좋다, 킬링타임 무비, 무비 연기 좋다, 무비 연기]","[연기 좋다, 킬링타임 무비 연기, 무비 연기, 킬링타임 무비]",3


In [ ]:
# pickle로 저장
with open('비상선언_키워드1_v1.pickle', 'wb') as f:
    pickle.dump(text1_pp, f)
with open('비상선언_키워드2_v1.pickle', 'wb') as f:
    pickle.dump(text2_pp_v1, f)
with open('비상선언_키워드2_v2.pickle', 'wb') as f:
    pickle.dump(text2_pp_v2, f)
with open('비상선언_키워드3_v1.pickle', 'wb') as f:
    pickle.dump(text3_pp_v1, f)
with open('비상선언_키워드3_v2.pickle', 'wb') as f:
    pickle.dump(text3_pp_v2, f)